# Data Docs

A new way to get started with your data.
- overview of features
- schemas and validation
- copy paste, code to production
- interactive
- also if you don't know python it should be useable

Jupyter Notebooks are great for that.

### Every Table got a structured Class like which inherits from a Meta Class
You should define a metaclass for each Destination.
That makes it easy to use the mapping we provide for whatever datastore you use.

In [ ]:
from sqlalchemy import types
from sqlalchemy import create_engine

from mlrepricer import setup

class AzureTarget():
    """
    For each data destination like MSSQL define your own ParentClass.
    """
    def __init__(self):
        # dtypes
        self.textshort = types.NVARCHAR(length=40)
        self.textmiddle = types.NVARCHAR(length=400)
        self.textlong = types.NVARCHAR(length=4000)  # 4000 is max for nvarchar
        self.floaty = types.Float
        self.inty = types.INTEGER
        self.datey = types.DATE
        self.datetimey = types.DATETIME
        self.timey = types.TIME
        self.booly = types.BINARY
        
        # connection data, pls change it
        self.conn_data = setup.configs['AzureSQL']
        
    def connection_engine(self):
        pymssqltext = 'mssql+pymssql://{}@{}:{}@{}:{}/{}'.format(
            self.conn_data['username'], self.conn_data['hostname'], self.conn_data['password'],
            self.conn_data['host'], self.conn_data['port'],
            self.database)
        return create_engine(pymssqltext)
    
    @property
    def conn(self):
        return self.connection_engine()
    
    @property
    def dtypes(self):
        return dict(zip(self.mapping.keys(), [item[0] for item in list(
            self.mapping.values())]))
    
    @property
    def nullable(self):
        return dict(zip(self.mapping.keys(), [item[1] for item in list(
                self.mapping.values())]))
        

In [ ]:
class AzureSaleTaxdoo(AzureTarget):
    """
    For each table define a class which inherits from the Destination class.
    """
    table = 'taxdoo_sale'
    if_exists = 'append'
    eventdate = 'transaction_arrival_date'
    filterdict = {'transaction_type': ['Transfer']}
    
    @property
    def mapping(self):
        return {
            'transaction_arrival_date': (self.datey, False),
            'transaction_number': (self.textshort, False),
            'taxdoo_net_amount': (self.floaty, False),
            'taxdoo_vat_amount': (self.floaty, False),
            'transaction_type': (self.textshort, False),
            'currency': (self.textshort, False),
            'sent_from_country': (self.textshort, False),
            'sent_to_country': (self.textshort, False),
            'taxdoo_country_of_taxation': (self.textshort, False)}


In [ ]:
m = AzureSaleTaxdoo()

In [ ]:
import pandas as pd
result = pd.read_sql_table(m.table, m.conn)

In [ ]:
m.nullable